In [51]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

In [52]:
train_data_path = './data/train.csv'
test_data_path = './data/test.csv'

train_data = pd.read_csv(train_data_path)
test_data = pd.read_csv(test_data_path)

In [53]:
data_all = pd.concat([train_data, test_data], axis=0)
#Age 使用每一个Pclass中的年龄的中位数填充年龄的缺失值
age_median_map = data_all[['Age', 'Pclass']].dropna().groupby('Pclass').agg({'Age' : 'median'}).to_dict()
data_all.Age = data_all.Age.fillna(data_all.Pclass.map(age_median_map['Age']))
#Fare票价使用价格的平均数填充(这里能否也按照Pclass分组呢？)
data_all.Fare = data_all.Fare.fillna(data_all.Fare.dropna().mean())
#Embarked 港口空值用X填充
data_all.Embarked = data_all.Embarked.fillna('X')
#船舱号的空值用X填充，并且取所有舱号的首字符作为舱号
data_all.Cabin = data_all.Cabin.fillna('X').map(lambda x : x[0].strip())
data_all.Ticket = data_all.Ticket.fillna('X').map(lambda x:str(x).split()[0] if len(str(x).split()) > 1 else 'X')
data_all.Name = data_all.Name.map(lambda x : x.split(',')[0]) 

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  """Entry point for launching an IPython kernel.


In [54]:
#数值类型的特征不变，对于非数值类型的数据进行编码
numerical_cols = ['Survived', 'Age', 'SibSp', 'Parch', 'Fare', 'Pclass']
one_hot_encode_cols = ['Sex', 'Cabin', 'Embarked']
label_encode_cols = ['Ticket']

In [55]:
from sklearn.preprocessing import LabelEncoder
def label_encode(df):
    le = LabelEncoder()
    return le.fit_transform(df)

In [56]:
#get_dummies只对于非数值型数据使用，对于数值型使用需要先转成str
#LabelEncoder的fit的参数的shape必须是一维的，例如(12,),所以这里使用apply而不能直接fit_transform(data_all[label_encode_cols])
numerical_df = data_all[numerical_cols]
one_hot_encode_df = pd.get_dummies(data_all[one_hot_encode_cols])
label_encode_df = data_all[label_encode_cols].apply(label_encode)

data_all = pd.concat([numerical_df, one_hot_encode_df, label_encode_df], axis=1)
# data_all = pd.concat([numerical_df, one_hot_encode_df], axis=1)

In [57]:
from sklearn.linear_model import LogisticRegression
X_train, X_test = data_all[:train_data.shape[0]].drop(columns='Survived'), data_all[train_data.shape[0]:].drop(columns='Survived')
y_train = data_all[:train_data.shape[0]]['Survived']

In [58]:
log_model = LogisticRegression()
log_model.fit(X_train, y_train)
pred = log_model.predict(X_test)

In [59]:
#分类提交应该为整数
pred = pred.astype("int")

In [60]:
samp_sub_path = './data/sample_submission.csv'
submisson = pd.read_csv(samp_sub_path)
submisson['Survived'] = pred
submisson.to_csv('logistic_reg_no_name.csv', index=False)

In [49]:
pred

array([0, 1, 1, ..., 0, 1, 1])